In [2]:
import os
import torch
import torchaudio
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts
from TTS.tts.layers.xtts.tokenizer import VoiceBpeTokenizer

print("Loading model...")
config = XttsConfig()
config.load_json("./models/xtts/config.json")
model = Xtts.init_from_config(config)
model.tokenizer = VoiceBpeTokenizer(vocab_file="./models/xtts/vocab.json")
model.init_models()
model.load_checkpoint(config, checkpoint_dir="./models/xtts/", use_deepspeed=True)
# model.cuda()

print("Computing speaker latents...")
gpt_cond_latent, speaker_embedding = model.get_conditioning_latents(audio_path=["./reference_speaker.wav"])

print("Inference...")
out = model.inference(
    "I b'a fɛ ne k'i ladɔnniya o la wa ?",
    "bm",
    gpt_cond_latent,
    speaker_embedding,
    temperature=0.7, # Add custom parameters here
)
torchaudio.save("xtts.wav", torch.tensor(out["wav"]).unsqueeze(0), 24000)

Loading model...
Computing speaker latents...


RuntimeError: Failed to open the input "reference_speaker.wav" (No such file or directory).

In [29]:
import IPython

def to_speech(text, temperature=0.7, speaker_reference="./reference_speaker.wav"):
    print("Computing speaker latents...")
    gpt_cond_latent, speaker_embedding = model.get_conditioning_latents(audio_path=["./reference_speaker.wav"])

    print("Inference...")
    out = model.inference(
        text,
        "bm",
        gpt_cond_latent,
        speaker_embedding,
        temperature=1, # Add custom parameters here
    )

    return IPython.display.Audio(torch.tensor(out["wav"]).unsqueeze(0).numpy(), rate=24000)

In [ ]:
os.path.realpath(config.model_args.tokenizer_file)

In [31]:
to_speech("Mun bɛ se ka Changé jamana baarakɛlaw ka sara la?")

Computing speaker latents...
Inference...
<tokenizers.Tokenizer object at 0x7f42f461aa30> [bm]mun[SPACE]bɛ[SPACE]se[SPACE]ka[SPACE]changé[SPACE]jamana[SPACE]baarakɛlaw[SPACE]ka[SPACE]sara[SPACE]la?


In [32]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_folder(
    folder_path="/workspace/coqui-TTS/finetuning/bambara/models/bambara-tts",
    repo_id="oza75/bambara-tts",
    repo_type="model",
)

mel_stats.pth:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

model.pth:   0%|          | 0.00/5.64G [00:00<?, ?B/s]

dvae.pth:   0%|          | 0.00/211M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/oza75/bambara-tts/commit/4b6aadec141304c34027dc40466a932352b868a9', commit_message='Upload folder using huggingface_hub', commit_description='', oid='4b6aadec141304c34027dc40466a932352b868a9', pr_url=None, pr_revision=None, pr_num=None)